### Installing reqs and downloading examples

In [ ]:
# If package is not installed
! pip install git+https://github.com/salute-developers/GigaAM.git

In [ ]:
# Downloading wavs for examples
!wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/example.wav
!wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/long_example.wav

## Speech Recognition

In [ ]:
import os
import warnings
from typing import Dict

import gigaam

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
model = gigaam.load_model(
    "ctc",  # GigaAM-V2 CTC model
    fp16_encoder=True,  # to use fp16 encoder weights - GPU only
    use_flash=False,  # disable flash attention - colab does not support it
)
model.transcribe("example.wav")

100%|███████████████████████████████████████| 888M/888M [02:27<00:00, 6.29MiB/s]


'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

In [ ]:
model = gigaam.load_model(
    "rnnt",  # GigaAM-V2 RNNT model
    device="cpu",  # CPU-inference
)
model.transcribe("example.wav")

100%|███████████████████████████████████████| 892M/892M [02:34<00:00, 6.07MiB/s]


'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

### Long-Form Speech Recognition

In [ ]:
!pip install gigaam[longform]

* For long-form inference:
  * generate [Hugging Face API token](https://huggingface.co/docs/hub/security-tokens)
  * accept the conditions to access [pyannote/voice-activity-detection](https://huggingface.co/pyannote/voice-activity-detection) files and content
  * accept the conditions to access [pyannote/segmentation](https://huggingface.co/pyannote/segmentation) files and content

In [ ]:
os.environ["HF_TOKEN"] = "<HF_TOKEN>"

In [ ]:
model = gigaam.load_model("ctc", use_flash=False)
recognition_result = model.transcribe_longform("long_example.wav")

for utterance in recognition_result:
    transcription = utterance["transcription"]
    start, end = utterance["boundaries"]
    print(f"[{gigaam.format_time(start)} - {gigaam.format_time(end)}]: {transcription}")

[00:00:00 - 00:16:83]: вечерня отошла давно но в кельях тихо и темно уже и сам игумен строгий свои молитвы прекратил и кости ветхие склонил перекрестясь на одр убогий кругом и сон и тишина но церкви дверь отворена
[00:17:10 - 00:32:61]: трепещет луч лампады и тускло озаряет он и темную живопись икон и позлощенные оклады и раздается в тишине то тяжкий вздох то шепот важный и мрачно дремлет в вышине старинный свод
[00:32:95 - 00:49:33]: глухой и влажный стоят за клиросом чернец и грешник неподвижны оба и шепот их как глаз из гроба и грешник бледен как мертвец монах несчастный полно перестань
[00:49:82 - 01:05:74]: ужасна исповедь злодея заплачена тобою дань тому кто в злобе пламенея лукаво грешника блюдет и к вечной гибели ведет смирись опомнись время время раскаянье покров
[01:05:97 - 01:10:90]: я разрешу тебя грехов сложи мучительное бремя


## Emotion recognition

In [ ]:
model = gigaam.load_model("emo")
emotion2prob: Dict[str, int] = model.get_probs("example.wav")

print(", ".join([f"{emotion}: {prob:.3f}" for emotion, prob in emotion2prob.items()]))

100%|███████████████████████████████████████| 924M/924M [02:04<00:00, 7.78MiB/s]


angry: 0.000, sad: 0.002, neutral: 0.923, positive: 0.074


## GigaAM embeddings

In [ ]:
# audio-only pretrained encoder
model = gigaam.load_model("ssl", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

100%|███████████████████████████████████████| 887M/887M [03:21<00:00, 4.62MiB/s]


tensor([[[-0.2829,  0.3638,  0.4520,  ..., -0.4743, -0.4033, -0.2417],
         [ 0.1611, -0.5006, -0.0584,  ..., -0.6239, -0.2320, -0.2054],
         [-1.1849, -1.0029, -0.6111,  ..., -0.5137, -0.3737, -0.2654],
         ...,
         [ 0.0181, -0.3763, -0.8959,  ...,  0.1716,  0.0556,  0.1298],
         [ 0.2690, -0.0654, -0.5020,  ..., -1.4432, -1.4827, -1.4490],
         [-1.5650, -1.6693, -1.2834,  ...,  0.5117,  0.4839,  0.0136]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)


In [ ]:
# you also can embed audio with CTC- or RNNT-finetuned encoder
model = gigaam.load_model("ctc", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

tensor([[[-1.0334, -0.2841, -0.3606,  ..., -0.2859, -0.6947, -0.7006],
         [-0.4317, -0.0140, -0.9296,  ...,  0.1781,  0.2170, -0.0181],
         [-0.9221, -1.1284, -0.6389,  ..., -1.0664, -1.3304, -1.2421],
         ...,
         [ 0.5749,  0.5176, -0.0996,  ...,  1.7497,  1.8691,  2.1302],
         [-0.2919, -0.8087, -1.2554,  ..., -0.7942, -0.7634, -0.7938],
         [-1.8086, -2.1976, -2.4012,  ...,  0.8310,  1.0165,  1.0165]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)


## Export to ONNX

In [ ]:
onnx_dir = "onnx"
model_type = "rnnt" # or "ctc"

model = gigaam.load_model(
    model_type,
    fp16_encoder=False,  # only fp32 tensors
    use_flash=False,  # disable flash attention
)
model.to_onnx(dir_path=onnx_dir)

100%|███████████████████████████████████████| 892M/892M [03:20<00:00, 4.66MiB/s]


Succesfully ported onnx v2_rnnt_encoder to onnx/v2_rnnt_encoder.onnx.
Succesfully ported onnx v2_rnnt_decoder to onnx/v2_rnnt_decoder.onnx.
Succesfully ported onnx v2_rnnt_joint to onnx/v2_rnnt_joint.onnx.


### ONNX inference

In [ ]:
from gigaam.onnx_utils import load_onnx_sessions, transcribe_sample

sessions = load_onnx_sessions(onnx_dir, model_type)
transcribe_sample("example.wav", model_type, sessions)

'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'